In [1]:
!ping https://8080-u6yuvi-dlpackage-81wykj58c1m.ws-us107.gitpod.io/embeddings

ping: https://8080-u6yuvi-dlpackage-81wykj58c1m.ws-us107.gitpod.io/embeddings: Name or service not known


In [ ]:
!pip3 install openai

## Embeddings Model

In [15]:
from openai import OpenAI

client = OpenAI(base_url="https://8080-u6yuvi-dlpackage-81wykj58c1m.ws-us107.gitpod.io/v1", api_key="NONE")

response = client.embeddings.create(
        input="Your text string goes here",
        model="text-embedding-ada-002"
)

print(response.data[0].embedding)

[-0.01277886, 0.0003618376, 0.021082528, 0.008389424, -0.009944025, -0.028036406, 0.026727995, 0.018707883, 0.042219914, 0.015701378, 0.013431587, 0.020911891, -0.000783515, -0.006226429, -0.013455435, -0.015460594, -0.0041951537, -0.028453354, -0.048900504, 0.015329604, -0.037667375, 0.002740343, -0.0890624, -0.015300789, -0.016249621, 0.020865064, 0.021733338, -0.003034033, 0.051191527, 0.05219854, -0.024775447, 0.00784956, 0.02333624, -0.034116957, -0.030624468, -0.012019045, 0.02744909, -0.032539196, -0.045496713, -0.0207107, -0.0019117343, -0.012825066, 0.009994428, -0.018718047, -0.05749402, -0.0009775048, -0.016141059, -0.01186634, 0.027391184, -0.0071935956, 0.01767488, -0.0052455063, 0.03122986, 0.010987814, 0.0020120682, 0.009648499, -0.022729468, 0.0016308816, -0.045427766, 0.023297284, 0.018285196, 0.03513919, 0.012977517, -0.036106195, 0.027266508, 0.021970272, 0.0038531185, -0.014898343, 0.037784748, -0.019791253, -0.0036172555, 0.01164206, 0.010963118, -0.024138944, -0.0

In [14]:
! curl https://8080-u6yuvi-dlpackage-81wykj58c1m.ws-us107.gitpod.io/v1 -X POST -H "Content-Type: application/json" -d '{"input": "Your text string goes here","model": "text-embedding-ada-002"}'

{"error":{"code":404,"message":"Cannot POST /v1","type":""}}

# Chat-Model

In [17]:
from openai import OpenAI

client = OpenAI(base_url="https://8080-u6yuvi-dlpackage-81wykj58c1m.ws-us107.gitpod.io/v1", api_key="NONE")

response = client.chat.completions.create(
        model="mistral",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Who won the world series in 2020?"},
            {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
            {"role": "user", "content": "Where was it played?"}
        ]
)
print(response)

ChatCompletion(id='91b77d8f-9fb1-435a-b97b-0015dba3bfc5', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The 2020 World Series was played at Arlington, Texas, at the Globe Life Field and the American Airlines Center.\n', role='assistant', function_call=None, tool_calls=None))], created=1706297149, model='mistral', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=0, prompt_tokens=0, total_tokens=0))


# RAG Pipeline

In [18]:
! pip install pypdf2 rich

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.3 MB/s eta 0:00:00a 0:00:01


In [20]:
!pip3 install numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 83.9 MB/s eta 0:00:00:00:0100:01


In [21]:
import io
import json
import uuid

import openai
from openai import OpenAI
import PyPDF2
import requests

import numpy as np

from rich.console import Console
from rich.progress import track, Progress
from pathlib import Path
from functools import cache

from prompts import DEFAULT_QUERY_PROMPT_TMPL

In [22]:
console = Console()
client = OpenAI(base_url="https://8080-u6yuvi-dlpackage-81wykj58c1m.ws-us107.gitpod.io/v1", api_key="NONE")

In [23]:
@cache
def get_embedding(text):
    response = client.embeddings.create(
		input=text,
		model="text-embedding-ada-002"
	)
    
    return response.data[0].embedding

In [24]:
def query_model(prompt):
	response = client.chat.completions.create(
		model="mistral",
		messages=[{"role": "user", "content": prompt}],
		temperature=0.7,
		max_tokens=512,
		top_p=1,
		frequency_penalty=0,
		presence_penalty=0
	)
	
	return response.choices[0].message.content

In [25]:
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [26]:
! wget https://arxiv.org/pdf/1706.03762.pdf

--2024-01-26 19:41:53--  https://arxiv.org/pdf/1706.03762.pdf
Resolving arxiv.org (arxiv.org)... 151.101.3.42, 151.101.67.42, 151.101.195.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.3.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2215244 (2.1M) [application/pdf]
Saving to: ‘1706.03762.pdf’

1706.03762.pdf      100%[===================>]   2.11M  --.-KB/s    in 0.05s   

2024-01-26 19:41:53 (42.5 MB/s) - ‘1706.03762.pdf’ saved [2215244/2215244]



In [27]:
with open("1706.03762.pdf", "rb") as f:
    pdf = PyPDF2.PdfReader(f)
    
    num_pages = len(pdf.pages)
    console.log(f"{num_pages=}")
    
    embeddings = []
    pdf_texts = []
    for page_num in track(range(num_pages), description="creating embeddings..."):
        pdf_text = pdf.pages[page_num].extract_text()
        embeddings.append(
            get_embedding(pdf_text)
        )
        pdf_texts.append(pdf_text)
    embeddings = np.array(embeddings)

[19:42:03] num_pages=15                                                                             ]8;id=830653;file:///tmp/ipykernel_4558/1154244257.py\1154244257.py]8;;\:]8;id=603770;file:///tmp/ipykernel_4558/1154244257.py#5\5]8;;\

Output()

In [28]:
console.log(pdf_texts[0])

[19:43:43] Provided proper attribution is provided, Google hereby grants permission to              ]8;id=966052;file:///tmp/ipykernel_4558/3549238774.py\3549238774.py]8;;\:]8;id=664100;file:///tmp/ipykernel_4558/3549238774.py#1\1]8;;\
           reproduce the tables and figures in this paper solely for use in journalistic or                        
           scholarly works.                                                                                        
           Attention Is All You Need                                                                               
           Ashish Vaswani∗                                                                                         
           Google Brain                                                                                            
           avaswani@google.comNoam Shazeer∗                                                                        
           Google Brain                                                                                            
           noam@google.comNiki Parmar∗                                                                             
           Google Research                                                                                         
           nikip@google.comJakob Uszkoreit∗                                                                        
           Google Research                                                                                         
           usz@google.com                                                                                          
           Llion Jones∗                                                                                            
           Google Research                                                                                         
           llion@google.comAidan N. Gomez∗ †                                                                       
           University of Toronto                                                                                   
           aidan@cs.toronto.eduŁukasz Kaiser∗                                                                      
           Google Brain                                                                                            
           lukaszkaiser@google.com                                                                                 
           Illia Polosukhin∗ ‡                                                                                     
           illia.polosukhin@gmail.com                                                                              
           Abstract                                                                                                
           The dominant sequence transduction models are based on complex recurrent or                             
           convolutional neural networks that include an encoder and a decoder. The best                           
           performing models also connect the encoder and decoder through an attention                             
           mechanism. We propose a new simple network architecture, the Transformer,                               
           based solely on attention mechanisms, dispensing with recurrence and convolutions                       
           entirely. Experiments on two machine translation tasks show these models to                             
           be superior in quality while being more parallelizable and requiring significantly                      
           less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-                               
           to-German translation task, improving over the existing best results, including                         
           ensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task,                          
           our model establishes a new singl

In [29]:
console.log(embeddings.shape)

[19:43:55] (15, 1024)                                                                               ]8;id=252315;file:///tmp/ipykernel_4558/3168160088.py\3168160088.py]8;;\:]8;id=362338;file:///tmp/ipykernel_4558/3168160088.py#1\1]8;;\

In [30]:
question = "what are key query values? can you elaborate more on it, in detail please?"
question_emb = get_embedding(question)

In [31]:
def query(query_text, text_embedding, doc_embeddings):
    relevant_emb_idx = np.argmax([
      cosine_similarity(text_embedding, doc_emb) for doc_emb in doc_embeddings
    ])
    
    relevant_text = pdf_texts[relevant_emb_idx]
    
    query_text = DEFAULT_QUERY_PROMPT_TMPL.substitute(
      context_str=relevant_text,
      query_str=query_text
    )
    
    return query_model(prompt=query_text)

In [32]:
answer = query(
    question,
    question_emb,
    embeddings
)

In [34]:
console.log(answer)

[19:47:31] Key query values are the specific elements or features that are being searched or        ]8;id=483357;file:///tmp/ipykernel_4558/2606236125.py\2606236125.py]8;;\:]8;id=373049;file:///tmp/ipykernel_4558/2606236125.py#1\1]8;;\
           retrieved in a database or information system. These values can be used to filter, sort,                
           or otherwise manipulate the data in a database or information system.                                   
                                                                                                                   
           For example, in a database with customer information, key query values could be the                     
           customer's name, address, phone number, or other identifying information. In a search                   
           engine, key query values could be the search terms or keywords that a user enters to                    
           find specific information.                                                                              
                                                                                                                   
           In a more detailed explanation, key query values can be broken down into the following                  
           categories:                                                                                             
                                                                                                                   
           1. Search Terms or Keywords: These are the words or phrases that a user enters into a                   
           search engine or database to find specific information.                                                 
                                                                                                                   
           2. Filtering Criteria: These are the specific attributes or features that a user wants                  
           to filter or sort the data by. For example, in a customer database, a user might want to                
           filter by location, age, or gender.                                                                     
                                                                                                                   
           3. Sorting Criteria: These are the specific attributes or features that a user wants to                 
           sort the data by. For example, in a customer database, a user might want to sort by the                 
           last name, age, or number of transactions.                                                              
                                                                                                                   
           4. Numeric Values: These are the specific numeric values that a user wants to search for                
           or filter by. For example, in a customer database, a user might want to search for                      
           customers with a specific age range or number of transactions.                                          
                                                                                                                   
           5. Date or Time Ranges: These are the specific date or time ranges that a user wants to                 
           search for or filter by. For example, in a customer database, a user might want to                      
           search for customers with a specific date of birth or time of their last transaction.                   
                                                                                                                   
           6. Geographic Locations: These are the specific geographic locations that a user wants                  
           to search for or filter by. For example, in a customer database, a user might want to                   
           filter by a specific city or coun

In [35]:
## Putting it all together

In [ ]:
CACHE_DIR = "./cache"
PDF_URL = "<https://arxiv.org/pdf/1706.03762.pdf>"
Path(CACHE_DIR).mkdir(exist_ok=True)


In [ ]:
def query_pdf(pdf_url, query):
    pdf_filename = Path(pdf_url).name
    
    local_file = Path(CACHE_DIR) / pdf_filename
    
    # first get the pdf if not there
    if not local_file.exists():
        with console.status("[bold yellow]downloading pdf...", spinner="runner"):
            # download and save in cache dir
            pdf_res = requests.get(pdf_url, timeout=300)
            # cache the pdf
            local_file.write_bytes(pdf_res.content)

        console.log("[bold green]pdf downloaded!")
    
    # check if embeddings exists in cache
    embedding_file = local_file.parent / f"{local_file.stem}.embeddings.json"
    
    if embedding_file.exists():
        with console.status(f"[bold yellow]loading embedding index from cache...", spinner="balloon"):
            with embedding_file.open("r") as f:
                embeddings_index = json.load(f)
            console.log("[bold green]embedding index loaded from cache")
    else:
        with console.status(f"[bold yellow]computing embeddings and index...", spinner="earth"):
            with local_file.open("rb") as f:
                pdf = PyPDF2.PdfReader(f)
                # compute embeddings and save to disk
                num_pages = len(pdf.pages)
                console.log(f"got pages: {num_pages=}")
            
                embeddings_index = {
                    "sequence": [],
                    "docs_store": {},
                    "vecs_store": {}
                }
                
                for page_num in track(range(num_pages), description="querying embeddings..."):
                    page_uuid = str(uuid.uuid4())
                    pdf_text = pdf.pages[page_num].extract_text()
                    
                    embeddings_index['docs_store'][page_uuid] = {
                        "id": page_uuid,
                        "text": pdf_text,
                        "page_num": page_num
                    }
                    
                    text_embedding = get_embedding(pdf_text)

                    embeddings_index['vecs_store'][page_uuid] = {
                        "id": page_uuid,
                        "vector": text_embedding,
                    }
                    
                    embeddings_index['sequence'].append(page_uuid)
                    
                # save this index
                with embedding_file.open("w") as f:
                    json.dump(embeddings_index, f)

        console.log("[bold green]done computing embedding index")
        
    # create embeddings numpy
    np_index_to_uuid = []
    embeddings_np = []
    for emb_uuid, emb in embeddings_index['vecs_store'].items():
        np_index_to_uuid.append(emb_uuid)
        embeddings_np.append(emb['vector'])
    embeddings_np = np.array(embeddings_np)
    
    # get query embed
    query_emb = get_embedding(query)
    
    # find the closest text to the query in doc store
    with console.status(f"[bold yellow]finding relevant text in doc...", spinner="earth"):
        relevant_text = embeddings_index['docs_store'][
            np_index_to_uuid[
                np.argmax([
                    cosine_similarity(query_emb, emb) for emb in embeddings_np
                ])
            ]
        ]['text']
    console.log("[bold green]found relevant text from input")
    
    query_text = DEFAULT_QUERY_PROMPT_TMPL.substitute(
        context_str=relevant_text,
        query_str=query
    )
    
    with console.status(f"[bold yellow]quering the model..."):
        model_out = query_model(prompt=query_text)
    
    console.log(f"[bold yellow]Question: {query}")
    console.log(f"[bold yellow]Context: {local_file.name}")
    console.log(f"[bold green]Answer: {model_out}")

In [ ]:
query_pdf(
	pdf_url=PDF_URL,
	query="explain the attention mechanism"
)

## LLama Index

In [36]:
!pip install llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.6 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 17.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 86.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 148.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 141.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 86.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 156.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 75.7 MB/s eta 0:00:0000:01:

In [37]:
from llama_index.llms import OpenAILike, ChatMessage

In [38]:
model = OpenAILike(model="mistral", api_base="https://8080-u6yuvi-dlpackage-81wykj58c1m.ws-us107.gitpod.io/v1", api_key="NONE", timeout=600)

In [39]:
response = model.chat(messages=[ChatMessage(content="How are you?", role="user")], max_tokens=128)

In [40]:
response

ChatResponse(message=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content="\n\nI'm doing well, thank you for asking! How can I help you today?<dummy32000>", additional_kwargs={}), raw={'id': '5c4db253-46ae-49c0-babf-705c21735f21', 'choices': [CompletionChoice(finish_reason='stop', index=0, logprobs=None, text="\n\nI'm doing well, thank you for asking! How can I help you today?<dummy32000>")], 'created': 1706297149, 'model': 'mistral', 'object': 'text_completion', 'system_fingerprint': None, 'usage': CompletionUsage(completion_tokens=0, prompt_tokens=0, total_tokens=0)}, delta=None, additional_kwargs={})

## Creating a text summarizer using LLM and Llama Index

## Create Embeddings for Vector Database

In [41]:
import os

from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader, LLMPredictor, PromptHelper, ServiceContext, VectorStoreIndex
from llama_index import StorageContext, load_index_from_storage
from llama_index.llms import OpenAI
from llama_index.llms import OpenAILike

# Uncomment to specify your OpenAI API key here (local testing only, not in production!), or add corresponding environment variable (recommended)
os.environ['OPENAI_API_KEY']= "NONE"
os.environ['OPENAI_API_BASE']= "https://8080-u6yuvi-dlpackage-81wykj58c1m.ws-us107.gitpod.io/v1"

embed_predictor = OpenAILike(model_name="text-embedding-ada-002")

In [43]:
max_input_size = 400
num_output = 400
max_chunk_overlap = 0.2

prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)


In [46]:
documents = SimpleDirectoryReader('data').load_data()

In [47]:
print(documents[0].text[:100])

Harry Potter and the Sorcerer's Stone


CHAPTER ONE

THE BOY WHO LIVED

Mr. and Mrs. Dursley, of num


In [48]:
len(documents[0].text)

5447

In [49]:
service_context = ServiceContext.from_defaults(llm=embed_predictor, prompt_helper=prompt_helper, chunk_size_limit=512)

In [50]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [51]:

index.storage_context.persist(persist_dir="./storage_small")

## Create Summarizer

In [54]:
chat_predictor = OpenAILike(model="mistral", timeout=120)

In [55]:
max_input_size = 1024
num_output = 256
max_chunk_overlap = 0.2

prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)

In [56]:
service_context = ServiceContext.from_defaults(llm=chat_predictor, prompt_helper=prompt_helper)

In [57]:
storage_context = StorageContext.from_defaults(persist_dir='./storage_small')

In [58]:
index = load_index_from_storage(storage_context, service_context=service_context)

In [59]:
query_engine = index.as_query_engine(response_mode="simple_summarize")

In [60]:
data = input("Question: ")

Question:  Write a short note


In [61]:
response = query_engine.query(data)

In [63]:
response.response

"\n\nIn the given text, the story revolves around Mr. Dursley, who is a character in J.K. Rowling's Harry Potter and the Philosopher's Stone. The text describes Mr. Dursley's daily routine, his thoughts, and his interactions with his family. The story highlights the mundane life of the Dursley family, who are the non-magical relatives of Harry Potter, the protagonist of the series. The text also mentions a mysterious owl, which is a hint towards the magical world that Harry Potter is about to enter.<dummy32000>"

In [ ]:
#top-2
query_engine = index.as_query_engine(
    response_mode="compact",
    similarity_top_k=2
)

In [ ]:
response = query_engine.query(data)

In [ ]:
query_engine = index.as_query_engine(
    response_mode="tree_summarize",
    similarity_top_k=2
)
response = query_engine.query(data)
print(response.response)

## Llama Hub

In [64]:
from llama_index import download_loader
from pathlib import Path
from llama_index import (
    VectorStoreIndex,
    get_response_synthesizer,
)
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.postprocessor import SimilarityPostprocessor

In [65]:
PDFReader = download_loader("PDFReader")

pdf_loader = PDFReader()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.9/283.9 kB 2.9 MB/s eta 0:00:00a 0:00:01


In [67]:
! wget https://arxiv.org/pdf/1706.03762.pdf

--2024-01-26 20:30:34--  https://arxiv.org/pdf/1706.03762.pdf
Resolving arxiv.org (arxiv.org)... 151.101.3.42, 151.101.131.42, 151.101.195.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.3.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2215244 (2.1M) [application/pdf]
Saving to: ‘1706.03762.pdf.1’

1706.03762.pdf.1    100%[===================>]   2.11M  --.-KB/s    in 0.04s   

2024-01-26 20:30:34 (49.2 MB/s) - ‘1706.03762.pdf.1’ saved [2215244/2215244]



In [68]:
documents = pdf_loader.load_data(file=Path('./1706.03762.pdf'))

In [69]:
documents[0]

Document(id_='594b4f35-4673-4231-a607-dce2fcf8c247', embedding=None, metadata={'page_label': '1', 'file_name': '1706.03762.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperform

In [70]:
chat_predictor = OpenAILike(model="mistral", timeout=120)
embed_predictor = OpenAILike(model_name="text-embedding-ada-002")

In [71]:
service_context_index = ServiceContext.from_defaults(llm=embed_predictor, prompt_helper=prompt_helper)

In [72]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context_index)

In [73]:
# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=1,
)

In [74]:
service_context_response = ServiceContext.from_defaults(llm=chat_predictor, prompt_helper=prompt_helper)


In [75]:
# configure response synthesizer
response_synthesizer = get_response_synthesizer(service_context=service_context_response)

In [76]:
# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
)

In [77]:
response = query_engine.query("What does Keys, Queries and Values mean?")

APITimeoutError: Request timed out.

In [78]:
print(response.response)



In the given text, the story revolves around Mr. Dursley, who is a character in J.K. Rowling's Harry Potter and the Philosopher's Stone. The text describes Mr. Dursley's daily routine, his thoughts, and his interactions with his family. The story highlights the mundane life of the Dursley family, who are the non-magical relatives of Harry Potter, the protagonist of the series. The text also mentions a mysterious owl, which is a hint towards the magical world that Harry Potter is about to enter.<dummy32000>


# Index as Chat Engine

In [ ]:
chat_engine = index.as_chat_engine()
response = chat_engine.chat("Tell me a joke.")

In [ ]:
#debugging 
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
query_engine = index.as_query_engine(
    similarity_top_k=3,
    # only works for pinecone
    vector_store_kwargs={
        "filter": {"name": "paul graham"},
    },
)
response = query_engine.query("what did the author do growing up?")